In [1]:
import torch.nn as nn
from torchvision import models
import torch
import torchvision.transforms as transforms
from PIL import Image
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import importlib
import Entity
from Entity import CustomDataset


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")  # GPUデバイスを取得
else:
    device = torch.device("cpu")  # CPUデバイスを取得

In [3]:
from learning.Entity.CustomDataset import PokemonMoveDataset


dataset = PokemonMoveDataset('D:/tanaka\Documents\poke-move\data\my-dataset\dataset_without_low_count_move.txt', device)

In [4]:
from torch.utils.data import DataLoader
learning_rate = 1e-5
batch_size = 16
epochs = 10
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

torch.manual_seed(42)
train_dataset, test_dataset = torch.utils.data.random_split(
    dataset, [train_size, val_size]
)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [5]:
import torch.nn.functional as F
import sys
import os
from datetime import datetime
from learning.Encoder.PokeMoveEncoder import PokeMoveEncoder

from learning.LossFunctions.ContractiveLoss import ContrastiveLoss
from learning.LossFunctions.WeightedLoss import WeightedLoss


model = PokeMoveEncoder(device).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# margin変更
loss_fn = WeightedLoss(device=device)

     182      1225
0    164  0.996735
1    156  0.995102
2    214  0.994286
3    263  0.986122
4    203  0.906122
..   ...       ...
892  896  0.000000
893  897  0.000000
894  898  0.000000
895  899  0.000000
896  900  0.000000

[897 rows x 2 columns]


In [10]:
def train_loop(dataloader, model: PokeMoveEncoder, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (pokemon, move, label, move_ids) in enumerate(dataloader):        
        # 予測と損失の計算
        label = label.to(device).float()
        pred = model(pokemon, move)
        # ここ不安
        loss = loss_fn(pred, label, move_ids).mean()
        # バックプロパゲーション
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 30 == 0:
            loss, current = loss.item() / len(pokemon), batch * len(label)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        if batch % 1000 == 0:
            now = datetime.now()
            # YYYY-MM-DD形式で日付を出力します
            formatted_date = now.strftime("%Y-%m-%d")
            torch.save(model.state_dict(), f'./model_{formatted_date}_add_loss.pth')

def test_loop(dataloader, model: PokeMoveEncoder, loss_fn):
    size = len(dataloader.dataset)
    test_loss = 0

    with torch.no_grad():
        for (pokemon, move, label, _) in dataloader:
            label = label.to(device).float()
            pred = model(pokemon, move)
            loss = loss_fn(pred, label).mean()
            test_loss += loss.item()

    test_loss /= size
    print(f"Avg loss: {test_loss:>8f} \n")

In [11]:
print("start")
for t in range(epochs):
    print(f"Epoch {t+1}\-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)

print("Done!")

start
Epoch 1\-------------------------------
tensor(0.1993, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: 0.033216  [    0/260983]
tensor(0.0906, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
tensor(0.1458, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
tensor(0.1302, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
tensor(0.1466, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
tensor(0.1152, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
tensor(0.1298, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
tensor(0.1100, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
tensor(0.1191, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
tensor(0.1158, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
tensor(0.1002, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
tensor(0.1160, device='cuda:0', dtype=torch.float64, grad_fn=<Mea

KeyboardInterrupt: 

In [ ]:
loss_fn.loss_weight

{164: tensor(0.9967, device='cuda:0', dtype=torch.float64),
 156: tensor(0.9951, device='cuda:0', dtype=torch.float64),
 214: tensor(0.9943, device='cuda:0', dtype=torch.float64),
 263: tensor(0.9861, device='cuda:0', dtype=torch.float64),
 203: tensor(0.9061, device='cuda:0', dtype=torch.float64),
 173: tensor(0.8743, device='cuda:0', dtype=torch.float64),
 496: tensor(0.8718, device='cuda:0', dtype=torch.float64),
 207: tensor(0.7902, device='cuda:0', dtype=torch.float64),
 92: tensor(0.7829, device='cuda:0', dtype=torch.float64),
 104: tensor(0.7747, device='cuda:0', dtype=torch.float64),
 237: tensor(0.7706, device='cuda:0', dtype=torch.float64),
 590: tensor(0.7706, device='cuda:0', dtype=torch.float64),
 216: tensor(0.7690, device='cuda:0', dtype=torch.float64),
 218: tensor(0.7690, device='cuda:0', dtype=torch.float64),
 213: tensor(0.7478, device='cuda:0', dtype=torch.float64),
 240: tensor(0.6702, device='cuda:0', dtype=torch.float64),
 241: tensor(0.6571, device='cuda:0', dty